### Optimización

$$argmin_{q} | ( \mathbf{q} - \mathbf{q_{0}} ) \oslash  \mathbf{q_{0}} |^{2} \;,\; | ( \mathbf{q} - \mathbf{q_{0}} ) \oslash  \mathbf{q_{0}} |^{\infty}
\\subject\to: \quad {Rq = b \;, \\ Mq \leq d}$$


Testing different solvers and parameters using cvxpy 

In [3]:
import cvxpy as cp
import numpy as np
import time

In [4]:
def evaluate_sol(x):
    """Recieves solution vector and computes difference w.r.t PRESUPUESTO and HISTORICO"""

    global diffs_prod, diffs_hist
    assert np.all(x > 0)
    
    diffs_prod = [sum([x[n] for n in Q[i,:,:].flatten()]) - PRES[i] for i in range(p)]
    
    diffs_hist = [x[n] - HIST[n] for n in range(t)]
    
    return np.around([np.sum(np.abs(diffs_prod)), np.max(diffs_prod), np.min(diffs_prod), np.mean(diffs_prod), 
                      np.std(diffs_prod), np.mean(diffs_hist), np.std(diffs_hist)], 4)

In [5]:
def solve(p, d, c, PRES, HIST, solver = 'SCS', o = None, _ = 1):
    # Vamos a optimizar para producto, depósito y canal
    # Hay aprox 139 productos, 17 depósitos y 3 canales
    
    # Vector con p*d*c variables
    global x, Q

    cons = []

    # todo elemento es mayor a 0
    for e in x:
        cons.append(e >= 0)

    # para el producto i:
    # sum(Q_idc) <= PRES[i]
    if _:
        for i in range(p):
            cons.append(sum([x[n] for n in Q[i,:,:].flatten()]) <= PRES[i])
    else:
        for i in range(p):
            cons.append(sum([x[n] for n in Q[i,:,:].flatten()]) == PRES[i])
    
    # funcion objetivo, minimizar la diferencia con el mes anterior (HIST) 
    # y maximizar la asignación de productos 

    if o:
        obj = cp.Minimize(o)
    else: 
        obj = cp.Minimize(cp.sum_squares(x - HIST) - sum(x))

    prob = cp.Problem(obj, cons)
    
    t0 = time.time()
    sol = prob.solve('SCS')
    t1 = time.time()
    
    S = np.fromiter(x.value, dtype = 'float32')
    
    _ = evaluate_sol(S)
    
    return list(_) + [np.around(t1-t0), prob.status]

In [6]:
np.random.seed(101)
p, d, c = 139, 17, 3
t = p*d*c
PRES = np.random.randint(10000, 20000, size = p)
HIST = np.floor(np.random.normal(PRES.sum()//t, size = t))

# Tensor con una var para cada producto+deposito+canal
Q = np.arange(t).reshape((p,d,c))
x = cp.Variable(t)


solvers =  ['CVXOPT', 'GLPK', 'GLPK_MI', 'SCS', 'LS']

objs = [cp.sum_squares(x - HIST), cp.sum_squares(x - HIST) - sum(x), cp.sum_squares(x - HIST) - 2*sum(x), 
        cp.sum_squares(x - HIST) - 3*sum(x), cp.sum_squares(x - HIST) - cp.sum_squares(x)]

In [ ]:
%%time

d = {}
for m in solvers:
    for j in range(len(objs)):
        for z in [0, 1]:
            try:
                d[(m, str(j), ['==', '<='][z])] = solve(p, d, c, PRES, HIST, m, objs[j], z)
            except:
                d[(m, str(j), ['==', '<='][z])] = None

In [ ]:
import pickle

path = input('out')
with open(path + '.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pandas as pd

for k in d:
    d[str(k)] = d[k]
    del d[k]
df = pd.DataFrame.from_dict(d)

In [19]:
s = solve(p, d, c, PRES, HIST)

In [69]:
df

,"(SCS, 3, ==)","(SCS, 3, <=)","(SCS, 4, ==)","(SCS, 4, <=)","(LS, 0, ==)","(LS, 0, <=)","(LS, 1, ==)","(LS, 1, <=)","(LS, 2, ==)","(LS, 2, <=)",...,"('GLPK_MI', '3', '==')","('GLPK_MI', '3', '<=')","('GLPK_MI', '4', '==')","('GLPK_MI', '4', '<=')","('SCS', '0', '==')","('SCS', '0', '<=')","('SCS', '1', '==')","('SCS', '1', '<=')","('SCS', '2', '==')","('SCS', '2', '<=')"
ΔPRESsum,None,None,None,None,2769.46,148865,1002.66,67417.4,687.274,8971.06,...,None,None,None,None,2769.46,148865,1002.66,67417.4,687.274,8971.06
ΔPRESmax,None,None,None,None,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548,...,None,None,None,None,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548
ΔPRESmin,None,None,None,None,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191,...,None,None,None,None,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191
ΔPRESmean,None,None,None,None,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54,...,None,None,None,None,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54
ΔPRESstd,None,None,None,None,22.3782,1502.37,6.2227,939.804,3.0349,186.104,...,None,None,None,None,22.3782,1502.37,6.2227,939.804,3.0349,186.104
ΔHISTmean,None,None,None,None,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282,...,None,None,None,None,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282
ΔHISTstd,None,None,None,None,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968,...,None,None,None,None,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968
t,None,None,None,None,42,42,47,47,46,47,...,None,None,None,None,42,41,45,45,46,46
status,None,None,None,None,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,...,None,None,None,None,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate


In [57]:
df.index = 'ΔPRESsum ΔPRESmax  ΔPRESmin  ΔPRESmean  ΔPRESstd  ΔHISTmean  ΔHISTstd  t status'.split()

In [ ]:
df.iloc[3, 4]

In [60]:
df.dropna(axis = 1)[:, df.iloc[3]]

,"(LS, 0, ==)","(LS, 0, <=)","(LS, 1, ==)","(LS, 1, <=)","(LS, 2, ==)","(LS, 2, <=)","('CVXOPT', '0', '==')","('CVXOPT', '0', '<=')","('CVXOPT', '1', '==')","('CVXOPT', '1', '<=')",...,"('GLPK_MI', '1', '==')","('GLPK_MI', '1', '<=')","('GLPK_MI', '2', '==')","('GLPK_MI', '2', '<=')","('SCS', '0', '==')","('SCS', '0', '<=')","('SCS', '1', '==')","('SCS', '1', '<=')","('SCS', '2', '==')","('SCS', '2', '<=')"
ΔPRESsum,2769.46,148865,1002.66,67417.4,687.274,8971.06,2769.46,148865,1002.66,67417.4,...,1002.66,67417.4,687.274,8971.06,2769.46,148865,1002.66,67417.4,687.274,8971.06
ΔPRESmax,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548,32.6655,-3.3467,5.0683,-2.4308,...,5.0683,-2.4308,0.4196,-1.548,32.6655,-3.3467,5.0683,-2.4308,0.4196,-1.548
ΔPRESmin,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191,-45.921,-4595.31,-16.7841,-3044.69,...,-16.7841,-3044.69,-10.2376,-857.191,-45.921,-4595.31,-16.7841,-3044.69,-10.2376,-857.191
ΔPRESmean,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54,-6.6363,-1070.97,-5.8607,-485.017,...,-5.8607,-485.017,-4.9102,-64.54,-6.6363,-1070.97,-5.8607,-485.017,-4.9102,-64.54
ΔPRESstd,22.3782,1502.37,6.2227,939.804,3.0349,186.104,22.3782,1502.37,6.2227,939.804,...,6.2227,939.804,3.0349,186.104,22.3782,1502.37,6.2227,939.804,3.0349,186.104
ΔHISTmean,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282,1.0071,-19.8622,1.0223,-8.3729,...,1.0223,-8.3729,1.041,-0.1282,1.0071,-19.8622,1.0223,-8.3729,1.041,-0.1282
ΔHISTstd,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968,55.5141,32.6094,55.1974,42.7819,...,55.1974,42.7819,55.1348,53.1968,55.5141,32.6094,55.1974,42.7819,55.1348,53.1968
t,42,42,47,47,46,47,46,49,61,61,...,46,43,45,45,42,41,45,45,46,46
status,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,...,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate,optimal_inaccurate
